# Train LASSO models

## Summary
This notebook is dedicated to training LASSO models to infer histone <-> metabolomics relationships.

## 1. Load  libraries

In [1]:
import os
import sys
import pickle
from joblib import dump, load
from dataclasses import dataclass, field

import pandas as pd
import numpy as np
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
import seaborn as sns

import keras_tuner as kt
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split

c:\Users\Scott\Envs\train_lasso\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Scott\Envs\train_lasso\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\Scott\Envs\train_lasso\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## 2. Load Data
This section performs standard data preprocessing for machine learning, as well as some data exploration. 

In [2]:
basepath = "D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/processed/model_inputs.xlsx"

### A. GCP

In [3]:
gcp_df = pd.read_excel(basepath, 'GCP_all', index_col=0)
gcp_df.head(3)

,H3K4me1,H3K4me2,H3K4ac1,H3K9me1,H3K9me2,H3K9me3,H3K9ac1,H3K14ac1,H3K18ac1,H3K23ac1,H3K36me1,H3K36me2,H3K36me3,H3K27me1,H3K27me2,H3K27me3,H3K27ac1,H3K56me1,H3K79me1,H3K79me2
ACH-000004,0.149556,0.843264,-0.008752,-0.158237,-0.509555,-0.965174,-0.736749,0.944949,-0.650464,0.449944,0.645977,0.882421,0.299744,-0.337517,0.437573,0.852723,0.163441,-0.244781,-0.294986,-0.302940
ACH-000007,-0.128835,-0.145601,-2.730630,0.540652,-0.086952,-0.106505,0.018602,0.905939,0.525775,0.037019,0.483093,0.283082,0.622002,0.414664,0.690137,0.210494,0.858167,-5.946821,0.000566,-0.294008
ACH-000008,0.671111,0.690180,-3.305619,0.158777,0.056590,0.325020,0.811277,0.140577,-0.007284,0.136998,-0.327871,-0.400730,-0.364356,0.143931,0.381972,0.515318,0.216003,1.185194,0.105189,0.119715


### B. Metabolomics

In [4]:
met_df = pd.read_excel(basepath, 'MET_nonsignaling', index_col=0)
met_df.head(3)

,2-aminoadipate,3-phosphoglycerate,alpha-glycerophosphate,4-pyridoxate,aconitate,adenine,adipate,alpha-ketoglutarate,AMP,citrate,...,heptanoylcarnitine,lauroylcarnitine,myristoylcarnitine,palmitoylcarnitine,stearoylcarnitine,oleylcarnitine,arachidonyl_carnitine,sarcosine,beta-alanine,anserine
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,-0.113363,0.174728,0.661087,-0.370089,-0.045246,0.434350,0.174760,-0.043661,1.249415,-0.213278,...,-0.487840,0.222715,0.947898,1.162742,0.784755,0.727978,0.297024,-0.350366,-0.215865,-0.154495
ACH-000007,-0.246379,-0.277360,-0.482734,-0.143775,0.042996,-0.544958,0.215370,0.003411,-0.258807,0.060592,...,-0.129539,-0.516559,-0.514631,-0.252187,-0.198568,-0.371144,0.280362,0.039629,0.191910,0.178265
ACH-000008,-0.301735,0.424280,0.549118,-0.056079,0.612689,-0.069253,-0.286402,0.373105,0.268231,0.521923,...,-0.056436,0.233915,-0.086494,-0.137966,-0.400825,-0.259514,-0.452228,-0.038606,0.253871,-0.314390


### C. AUC Values

In [5]:
auc_df = pd.read_excel(basepath, 'AUC_all', index_col=0)
auc_df.head(3)

,AGK-2,BIX-01294,BRD-A02303741,BRD-A94377914,BRD-K11533227,BRD-K24690302,BRD-K29313308,BRD-K51490254,BRD-K61166597,BRD-K66532283,...,apicidin,belinostat,entinostat,isonicotinohydroxamic acid,methylstat,pandacostat,salermide,tacedinaline,tubastatin A,vorinostat
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,14.877,13.529,14.585,11.594,12.278,13.570,14.566,17.518,8.9811,11.0480,...,9.434,8.88944,9.7167,15.9458,12.0298,14.211,14.1318,13.2304,14.602,10.233
ACH-000007,14.366,13.048,15.191,13.698,16.100,14.687,14.367,16.402,17.1980,15.3090,...,11.623,10.16470,11.2560,14.9078,12.5518,14.360,13.2092,13.9350,14.780,11.618
ACH-000008,14.895,14.382,14.658,13.832,14.713,14.547,14.804,14.866,14.0280,14.3488,...,10.995,11.71500,10.9320,14.9120,14.0930,14.922,15.0000,14.5500,14.556,12.997


Because the scale of values is interpretable, we need to perform min max scaling.

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_auc = scaler.fit_transform(auc_df)
scaled_auc = pd.DataFrame(scaled_auc, index=auc_df.index, columns=auc_df.columns)
scaled_auc.head(3)

,AGK-2,BIX-01294,BRD-A02303741,BRD-A94377914,BRD-K11533227,BRD-K24690302,BRD-K29313308,BRD-K51490254,BRD-K61166597,BRD-K66532283,...,apicidin,belinostat,entinostat,isonicotinohydroxamic acid,methylstat,pandacostat,salermide,tacedinaline,tubastatin A,vorinostat
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000004,0.531899,0.492188,0.462295,0.457760,0.410916,0.458200,0.668860,0.880701,0.310683,0.459457,...,0.425670,0.451989,0.513591,0.652583,0.754526,0.532256,0.664849,0.631698,0.672095,0.398489
ACH-000007,0.489138,0.434999,0.521337,0.698705,0.759982,0.580634,0.654161,0.733588,0.899345,0.799521,...,0.577507,0.579249,0.632186,0.503209,0.799077,0.555417,0.599672,0.706969,0.687942,0.503837
ACH-000008,0.533405,0.593608,0.469408,0.714050,0.633307,0.565289,0.686440,0.531110,0.672245,0.722889,...,0.533947,0.733956,0.607224,0.503813,0.930613,0.642779,0.726182,0.772669,0.667999,0.608729


## 3. Define necessary functions
First, we need to define functions that will build the LASSO model.

In [7]:
from scipy.stats import pearsonr

def calcPCC(model, xval, yval):
    """Calculate the PCC and p-value from the trained model"""
    yhat = model.predict(xval)
    return pearsonr(np.squeeze(yhat), yval)

def build_model(hp):
    """Wrapper to build naive LASSO model and tune L1 penalty and learning rate."""
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
    hp_l1_penalty = hp.Choice('l1_penalty', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    lasso = tf.keras.Sequential([
        tf.keras.layers.Dense(
                                units=1, 
                                kernel_regularizer=tf.keras.regularizers.L1(hp_l1_penalty),
        )
    ])

    lasso.compile(
        optimizer=tf.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='mse'
    )

    return lasso

def hyperparameter_tuning(X, y):
    """Get the best model from hyperparameter tuning using `build_model` wrapper."""
    from tensorflow.keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)

    tuner = kt.Hyperband(
                    build_model,
                    objective='val_loss',
                    max_epochs=1000,
                    factor=3,
    )

    tuner.search(X, y, epochs=1000, callbacks=[early_stopping])
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    return tuner.hypermodel.build(best_hps)

def train_models(input, output, model_savepath:str=None, results_savepath:str=None):
    """Train LASSO models for a single- or multi-target problem."""
    from tensorflow.keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)

    training_history = list()
    models = list()
    for target in output.columns:
        y = output[target].astype(float).values.astype('float32')
        X = input.values.astype('float32')

        Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=model_savepath+'/'+target,
                                                    verbose=1,
                                                    save_best_only=True
                                                    
        )
        
        model = hyperparameter_tuning(Xtrain, ytrain)
        history = model.fit(
                            Xtrain, ytrain, 
                            epochs=1000, 
                            validation_data=(Xtest, ytest), 
                            callbacks=[early_stopping, cp_callback]
                        )
        PCC, pvalue = calcPCC(model, Xtest, ytest)
        
        training_history.append(
                                {
                                    'history': history.history,
                                    'PCC': PCC,
                                    'pvalue': pvalue
                                }
        )
        models.append(model)
    
    with open(results_savepath, 'wb') as f:
        pickle.dump(training_history, f)

    return training_history, models

## 3. Metabolism and AUC predictions

### A. Metabolism to AUC

In [46]:
training_history, models = train_models(
                                        input=met_df, 
                                        output=scaled_auc,
                                        model_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2auc",
                                        results_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/met2auc_results.pkl"
)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
 1/16 [>.............................] - ETA: 3s - loss: 0.6713
Epoch 1: val_loss improved from inf to 0.15128, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2auc\AGK-2
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2auc\AGK-2\assets
16/16 [==============================] - 1s 47ms/step - loss: 0.2681 - val_loss: 0.1513
Epoch 2/1000
 1/16 [>.............................] - ETA: 0s - loss: 0.1103
Epoch 2: val_loss improved from 0.15128 to 0.07747, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2auc\AGK-2
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2auc\AGK-2\assets
16/16 [==============================] - 0s 32

Now we need to construct a matrix of the weights for each metabolite/AUC pair.

In [11]:
def construct_weights_df(input_df:pd.DataFrame, target_df:pd.DataFrame, models_list:list):
    """
    """
    weights_df = pd.DataFrame(
                                index=input_df.columns, 
                                columns=target_df.columns
                            )
    for idx, obj in enumerate(target_df.columns):
        weights_df[obj] = models_list[idx].layers[0].weights[0].numpy()
    
    return weights_df

In [29]:
auc_weights = construct_weights_df(met_df, scaled_auc, models)
auc_weights.to_csv("D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/auc_weights.csv")

### B. AUC to Metabolism

In [49]:
training_history, models = train_models(
                                        input=scaled_auc, 
                                        output=met_df,
                                        model_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/auc2met",
                                        results_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/auc2met_results.pkl"
)
met_weights = construct_weights_df(scaled_auc, met_df, models)
met_weights.to_csv("D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/met_weights.csv")

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
 1/16 [>.............................] - ETA: 3s - loss: 0.6283
Epoch 1: val_loss improved from inf to 0.11095, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/auc2met\2-aminoadipate
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/auc2met\2-aminoadipate\assets
16/16 [==============================] - 1s 54ms/step - loss: 0.1917 - val_loss: 0.1110
Epoch 2/1000
 1/16 [>.............................] - ETA: 0s - loss: 0.1379
Epoch 2: val_loss improved from 0.11095 to 0.08077, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/auc2met\2-aminoadipate
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/auc2met\2-aminoadipate\assets
16/16 [===

## 3. Metabolism and GCP Predictions

### A. Metabolism to GCP

In [9]:
training_history, models = train_models(
                                        input=met_df, 
                                        output=gcp_df,
                                        model_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2gcp",
                                        results_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/met2gcp_results.pkl"
)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
 1/16 [>.............................] - ETA: 3s - loss: 0.5405
Epoch 1: val_loss improved from inf to 0.32068, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2gcp\H3K4me1
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2gcp\H3K4me1\assets
16/16 [==============================] - 1s 31ms/step - loss: 0.4344 - val_loss: 0.3207
Epoch 2/1000
 1/16 [>.............................] - ETA: 0s - loss: 0.2816
Epoch 2: val_loss improved from 0.32068 to 0.25748, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2gcp\H3K4me1
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/met2gcp\H3K4me1\assets
16/16 [==============================]

In [12]:
gcp_weights = construct_weights_df(met_df, gcp_df, models)
gcp_weights.to_csv("D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/met2gcp_weights.csv")

### B. GCP to Metaboites

In [13]:
training_history, models = train_models(
                                        input=gcp_df, 
                                        output=met_df,
                                        model_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/gcp2met",
                                        results_savepath="D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/gcp2met_results.pkl"
)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
 1/16 [>.............................] - ETA: 2s - loss: 0.6260
Epoch 1: val_loss improved from inf to 0.39681, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/gcp2met\2-aminoadipate
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/gcp2met\2-aminoadipate\assets
16/16 [==============================] - 1s 31ms/step - loss: 0.4964 - val_loss: 0.3968
Epoch 2/1000
 1/16 [>.............................] - ETA: 0s - loss: 0.3592
Epoch 2: val_loss improved from 0.39681 to 0.27947, saving model to D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/gcp2met\2-aminoadipate
INFO:tensorflow:Assets written to: D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/models/gcp2met\2-aminoadipate\assets
16/16 [===

In [ ]:
met_weights = construct_weights_df(gcp_df, met_df, models)
met_weights.to_csv("D:/Chandrasekaran/Projects/Epigenetics-Metabolism/Data/results/LASSO/gcp2met_weights.csv")

## 4. 10-fold cross validation
This code block runs 10-fold cross validation.

In [9]:
from sklearn.model_selection import KFold
def make_dataset(x, y, n_splits):

    def gen():
        for train_index, test_index in KFold(n_splits).split(x):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            yield X_train, y_train, X_test, y_test

    return tf.data.Dataset.from_generator(gen, (tf.float64, tf.float64, tf.float64, tf.float64))


def test_iter(input, output, n_splits=5):
    """"""
    for target in output.columns:
        dataset = make_dataset(x=input.values.astype('float32'), 
                                y=output[target].astype(float).values.astype('float32'), 
                                n_splits=n_splits)
        #iter = dataset.make_initializable_iterator()
        return dataset

In [8]:
def calcPCC(model, xval, yval):
    """Calculate the PCC and p-value from the trained model"""
    yhat = model.predict(xval)
    return pearsonr(np.squeeze(yhat), yval)

def build_model(hp):
    """Wrapper to build naive LASSO model and tune L1 penalty and learning rate."""
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
    hp_l1_penalty = hp.Choice('l1_penalty', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])

    lasso = tf.keras.Sequential([
        tf.keras.layers.Dense(
                                units=1, 
                                kernel_regularizer=tf.keras.regularizers.L1(hp_l1_penalty),
        )
    ])

    lasso.compile(
        optimizer=tf.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='mse'
    )

    return lasso

def hyperparameter_tuning(X, y):
    """Get the best model from hyperparameter tuning using `build_model` wrapper."""
    from tensorflow.keras.callbacks import EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)

    tuner = kt.Hyperband(
                    build_model,
                    objective='loss',
                    max_epochs=1000,
                    factor=3,
    )

    tuner.search(X, y, epochs=1000, callbacks=[early_stopping])
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    return tuner.hypermodel.build(best_hps)

def kfold_cv(input, output, n_splits:int=5) -> pd.DataFrame:
    from tensorflow.keras.callbacks import EarlyStopping
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    
    training_history = list()
    models = list()
    
    for target in output.columns:
        dataset = make_dataset(x=input.values.astype('float32'), 
                                y=output[target].astype(float).values.astype('float32'), 
                                n_splits=n_splits)

        cv_pcc = list()
        cv_r2 = list()

        for Xtrain, ytrain, Xtest, ytest in dataset.take(n_splits):
                    
        
            model = hyperparameter_tuning(Xtrain, ytrain)
            history = model.fit(
                    Xtrain, ytrain, 
                    epochs=1000, 
                    validation_data=(Xtest, ytest), 
                    callbacks=[early_stopping]
                )
            PCC, pvalue = calcPCC(model, Xtest, ytest)
            R2 = PCC**2
            
            cv_pcc.append(PCC)
            cv_r2.append(R2)

        
        training_history.append(
                                {
                                    'Target':target,
                                    'PCC': sum(cv_pcc)/len(cv_pcc),
                                    'R2': sum(cv_r2)/len(cv_r2)
                                }
        )
    
    return pd.DataFrame(training_history)

In [10]:
results = kfold_cv(input=met_df, output=gcp_df)

INFO:tensorflow:Reloading Oracle from existing project .\untitled_project\oracle.json
INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/1000
16/16 [==============================] - 0s 8ms/step - loss: 0.4425 - val_loss: 0.2578
Epoch 2/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2808 - val_loss: 0.2027
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2603 - val_loss: 0.2136
Epoch 4/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2390 - val_loss: 0.2091
Epoch 5/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2325 - val_loss: 0.2078
Epoch 6/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2172 - val_loss: 0.2253
Epoch 7/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2138 - val_loss: 0.1966
Epoch 8/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2100 - val_loss: 0.2076
Epoch 9/

In [12]:
results.to_csv('C:/Users/Scott/Desktop/18-nov-2022_lasso_10xCV.csv')

# Summary
Now let's analyze the results.